# Film Sentiment Quickstart
End-to-end notebook: load data, train TF-IDF + Logistic Regression, save model, and test predictions/API.

In [ ]:
# Environment setup
import os, sys
# Ensure project root is on sys.path if running from notebooks/
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root not in sys.path: sys.path.append(root)
import pandas as pd
print('Python:', sys.version)

## Load Dataset

In [ ]:
from src.data_loader import load_dataset
csv_path = os.path.join(root, 'data', 'IMDB Dataset.csv')
df = load_dataset(csv_path)
df.head()

## Train Model and Save

In [ ]:
from src.training.train import train_model
import joblib
model, metrics = train_model(df, test_size=0.2, random_state=42)
print('Accuracy:', round(metrics['accuracy'], 4))
print('F1 (macro):', round(metrics['f1'], 4))
model_path = os.path.join(root, 'models', 'sentiment_model.joblib')
os.makedirs(os.path.dirname(model_path), exist_ok=True)
joblib.dump(model, model_path)
print('Saved model to', model_path)

## Quick Predictions

In [ ]:
texts = ["This was a great movie!", "Terrible plot and acting."]
preds = model.predict(texts)
labels = ['positive' if p == 1 else 'negative' for p in preds]
list(zip(texts, labels))

## Test FastAPI Endpoint (in-notebook)
Demonstrates calling the API app using FastAPI's TestClient without starting a server.

In [ ]:
# Ensure API loads the saved model via MODEL_PATH at import time
import importlib
os.environ['MODEL_PATH'] = model_path
if 'api.main' in sys.modules: del sys.modules['api.main']
import api.main as apimod
# Load model into API state (mirrors startup)
apimod.load_model()
from fastapi.testclient import TestClient
client = TestClient(apimod.app)
resp = client.post('/v1/sentiment', json={'texts': ['Absolutely fantastic!', 'Not my taste']})
resp.status_code, resp.json()

## Optional: Call a running server
If you run: `uvicorn api.main:app --reload` (after setting `MODEL_PATH`), you can use `requests` below.

In [ ]:
# import requests
# r = requests.post('http://127.0.0.1:8000/v1/sentiment', json={'texts': ['Great film']})
# r.json()